In [1]:
from pathlib import Path
import argparse
import json
import math
import os
import random
import signal
import subprocess
import sys
import time

from PIL import Image, ImageOps, ImageFilter
from torch import nn, optim
import torch
import torchvision
import torchvision.transforms as transforms
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights,MaskRCNN_ResNet50_FPN_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate
import miou_eval as mi

In [2]:
def combine_dims(a, start=0, count=2):
    """ Reshapes numpy array a by combining count dimensions, 
        starting at dimension index start """
    s = a.shape
    return np.reshape(a, s[:start] + (-1,) + s[start+count:])


def collate_fn(batch):
    return tuple(zip(*batch))

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
print(device)

cuda


In [4]:
class MMCellDataset(Dataset):
    def __init__(self,root_dir, tester = False, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.img = []
        self.nml = []
        self.normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.tester = tester
        self.datano =  root_dir.split('/')[1]
        print(self.datano)
        for im_name in os.listdir(self.root_dir):
            self.nml.append(im_name)
            # tmplist = []
            # lent = len(im_name[:-4])
            # print(im_name,im_name[:lent])
            # imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
            # tmplist.append(imgtmp[0])
            # tmplist.append(imgtmp[1])
            # tmplist.append(imgtmp[2])
            # for gt_name in os.listdir("../"+self.datano+"/ground_truths"):
            #     if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
            #         mask = np.array(cv2.imread("../"+self.datano+"/ground_truths/"+gt_name,0))
            #         tmplist.append(mask)
            # tmplist = np.array(tmplist)
            # #print(tmplist.shape)
            # self.img.append(tmplist)   
    
    def __len__(self):
        return len([name for name in os.listdir(self.root_dir)])

    def __getitem__(self, idx):
        seq = iaa.Sequential([
            iaa.WithHueAndSaturation([
            iaa.WithChannels(0, iaa.Add((-15, 15))),
            iaa.WithChannels(1, [
                iaa.Multiply((0.5, 1.5)),
                iaa.LinearContrast((0.5, 1.5))
                ])
            ])
            ,
            ])
        seq2 = iaa.Sequential([
        iaa.Fliplr(0.5), # horizontal flips
        iaa.Crop(percent=(0, 0.1)), # random crops
        # Small gaussian blur with random sigma between 0 and 0.5.
        # But we only blur about 50% of all images.
        iaa.Sometimes(
            0.5,
            iaa.GaussianBlur(sigma=(0, 0.5))
        ),
        # Strengthen or weaken the contrast in each image.
        iaa.LinearContrast((0.75, 1.5)),
        # Add gaussian noise.
        # For 50% of all images, we sample the noise once per pixel.
        # For the other 50% of all images, we sample the noise per pixel AND
        # channel. This can change the color (not only brightness) of the
        # pixels.
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Multiply((0.8, 1.2), per_channel=0.2),
        # Apply affine transformations to each image.
        # Scale/zoom them, translate/move them, rotate them and shear them.
    ], random_order=True) # apply augmenters in random order

        if torch.is_tensor(idx):
            idx = idx.tolist()
        im_name = self.nml[idx]
        tmplist = []
        lent = len(im_name[:-4])
        #print(im_name,im_name[:lent])
        imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
        tmplist.append(imgtmp[0])
        tmplist.append(imgtmp[1])
        tmplist.append(imgtmp[2])
        cytomask = np.zeros(imgtmp[0].shape)
        nucmask = np.zeros(imgtmp[0].shape)
        remmask = np.zeros(imgtmp[0].shape)
        for gt_name in os.listdir("../"+self.datano+"/ground_truths"):
            if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
                mask = np.array(cv2.imread("../"+self.datano+"/ground_truths/"+gt_name,0))
                cytomask += (mask==20).astype(np.uint)
                nucmask += (mask==40).astype(np.uint)
                remmask += (mask==0).astype(np.uint)
                tmplist.append(mask)
        tmplist = np.array(tmplist)
        #print("Tmplist: ",tmplist.shape)
        patch_stack = tmplist
        patch_stack = np.array(patch_stack)
        # k_w = random.randint(700,patch_stack.shape[2])
        # k_h = random.randint(700,patch_stack.shape[1])
        k_w = 1000
        k_h = 1000
        # k_w = patch_stack.shape[2]
        # k_h = patch_stack.shape[1]
        o_w = random.randint(0,patch_stack.shape[2]-k_w)
        o_h = random.randint(0,patch_stack.shape[1]-k_h)
        #print(cytomask.shape)
        cytomask = (cytomask!=0).astype(np.uint8)[o_h:o_h+k_h,o_w:o_w+k_w]
        nucmask = (nucmask!=0).astype(np.uint8)[o_h:o_h+k_h,o_w:o_w+k_w]
        remmask = (remmask!=0).astype(np.uint8)[o_h:o_h+k_h,o_w:o_w+k_w]
        patch_img = patch_stack[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
        tesc = np.array([patch_img[0]*cytomask,patch_img[1]*cytomask,patch_img[2]*cytomask]).transpose(1,2,0)
        #print(tesc.shape)
        cytoimg = seq(image = np.array([patch_img[0]*cytomask,patch_img[1]*cytomask,patch_img[2]*cytomask]).transpose(1,2,0))
        nucimg = seq(image = np.array([patch_img[0]*nucmask,patch_img[1]*nucmask,patch_img[2]*nucmask]).transpose(1,2,0))
        remimg = seq(image = np.array([patch_img[0]*remmask,patch_img[1]*remmask,patch_img[2]*remmask]).transpose(1,2,0))
        patch_img2 = seq2(image = cv2.cvtColor((cytoimg+nucimg+remimg), cv2.COLOR_BGR2LAB )).transpose(2,0,1)
        patch_img1 = seq2(image = cv2.cvtColor(patch_img.transpose(1,2,0), cv2.COLOR_BGR2LAB )).transpose(2,0,1)
        patch_img1 = torch.as_tensor(patch_img1, dtype=torch.float32)
        patch_img1 = patch_img1/255
        patch_img1 = self.normalize(patch_img1)
        patch_img2 = torch.as_tensor(patch_img2, dtype=torch.float32)
        patch_img2 = patch_img2/255
        patch_img2 = self.normalize(patch_img2)
        #instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        #instances = instances.transpose(1,2,0)
        # data = {}
        # masks = []
        # boxes = []
        # area = []
        # t=0
        # for a in range(instances.shape[2]):
        #     dispim = instances[:,:,a]
        #     if np.all(dispim == 0):
        #         continue
        #     x,y,w,h = cv2.boundingRect(dispim)
        #     boxes.append([x, y, x+w, y+h])
        #     area.append(torch.tensor(h*w))
        #     masks.append(dispim/255)
        #     t=1
        # if t==0:
        #     #print("Abort")
        #     if self.tester:
        #         return "Problem","Hao gai"
        #     else:
        #         #print("Abort Abort ")
        #         return self.__getitem__((idx+1)%len(self.nml))
        # masks = np.array(masks)
        # masks = torch.as_tensor(masks, dtype=torch.uint8)
        # boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # area = torch.as_tensor(area, dtype=torch.float32)
        # img = torch.as_tensor(patch_img, dtype=torch.float32)
        # img = img/255
        # data["boxes"] =  boxes
        # iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        # data["iscrowd"] = iscrowd
        # data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        # data["masks"] = masks
        # data["area"] = area
        # data["image_id"] = torch.tensor(idx)
        return torch.stack([patch_img1,patch_img2])
train = MMCellDataset("../bigWW/images/")
#test = MMCellDataset("../dataset12/test/")
fintest = MMCellDataset("../test/images/",True)
#train = torch.utils.data.ConcatDataset([train, test])
#print(len(test))
trainloader = DataLoader(train, batch_size=128, shuffle=True,num_workers=10)
#testloader = DataLoader(test, batch_size=1, shuffle=True,collate_fn = collate_fn,num_workers=10)
#fintestloader = DataLoader(fintest, batch_size=1, shuffle=True,collate_fn = collate_fn,num_workers=10)

bigWW
test


In [5]:
# print(train[0][1].shape)
# writer = cv2.cvtColor(train[0][1].transpose(1,2,0),cv2.COLOR_LAB2BGR)
# cv2.imwrite("problem.png",writer)
print(len(train))

498


In [6]:
class GaussianBlur(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            sigma = random.random() * 1.9 + 0.1
            return img.filter(ImageFilter.GaussianBlur(sigma))
        else:
            return img


class Solarization(object):
    def __init__(self, p):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            return ImageOps.solarize(img)
        else:
            return img


class Transform:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(224, interpolation=Image.BICUBIC),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4,
                                        saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
            GaussianBlur(p=1.0),
            Solarization(p=0.0),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        self.transform_prime = transforms.Compose([
            transforms.RandomResizedCrop(224, interpolation=Image.BICUBIC),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4,
                                        saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
            GaussianBlur(p=0.1),
            Solarization(p=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __call__(self, x):
        y1 = self.transform(x)
        y2 = self.transform_prime(x)
        return y1, y2


In [7]:
def off_diagonal(x):
    # return a flattened view of the off-diagonal elements of a square matrix
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

In [8]:
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor
class BarlowTwins(nn.Module):
    def __init__(self, args):
        super().__init__()
        #self.args = args
        self.backbone = torchvision.models.resnet50(zero_init_residual=True)
        #self.backbone = _resnet_fpn_extractor(self.backbone, 5)
        self.backbone.fc = nn.Identity()

        # projector
        sizes = [2048,8192] 
        layers = []
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=False))
            layers.append(nn.BatchNorm1d(sizes[i + 1]))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Linear(sizes[-2], sizes[-1], bias=False))
        self.projector = nn.Sequential(*layers)

        # normalization layer for the representations z1 and z2
        self.bn = nn.BatchNorm1d(sizes[-1], affine=False)

    def forward(self, y):
        y1 = y[0]
        y2 = y[1]
        z1 = self.projector(self.backbone(y1))
        z2 = self.projector(self.backbone(y2))

        # empirical cross-correlation matrix
        c = self.bn(z1).T @ self.bn(z2)

        # sum the cross-correlation matrix between all gpus
        c.div_(128)
        #torch.distributed.all_reduce(c)

        on_diag = torch.diagonal(c).add_(-1).pow_(2).sum()
        off_diag = off_diagonal(c).pow_(2).sum()
        loss = on_diag + 1 * off_diag
        return loss

In [9]:
# bt = BarlowTwins(None)
# print(bt(torch.stack([train[0][0],train[1][0]]),torch.stack([train[0][1],train[1][1]])))

from collections import OrderedDict


model = BarlowTwins(None)
model.to(device)
dicl = torch.load("./resenter2.1.torch")
#print(dicl.keys())
ml = OrderedDict()
for i in dicl.keys():
    if(i[0:4]=="body"):
        ml[i[5:]] = dicl[i]
    else:
        ml[i] = dicl[i]
#print(dicl)
model.backbone.load_state_dict(ml,strict=False)
#model.load_state_dict(torch.load("alchemy_0_9116.torch"))
# lr=0.1
# if(iv>=10==0):
#     lr=6e-5
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001,momentum=0,weight_decay=0.00004)
#optimizer = torch.optim.AdamW(model.parameters(), lr=8e-5)
model.train()
for epoch in range(1):
    for i, images in enumerate(trainloader, 0):

            #lrs+=1
            images = images.cuda()
            #images = list(image.to(device) for image in images)
            #targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            optimizer.zero_grad()
            loss = model(images)
            #losses = sum(loss for loss in loss_dict.values())
            #if losses.item()>=0.2:
            loss.backward()
            optimizer.step()
            del images
            #print("lr: ",optimizer.param_groups[0]['lr'])
            print(i,'loss:', loss.item())
            # if lrs==50:
            #     lrs=0
            #     #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 200)
            #     # for param_group in optimizer.param_groups:
            #     #     param_group['lr'] = lr*0.6
            #     #     lr = lr * 0.6
            #     val = evalm(model,fintestloader)
            #     if(val>min_val):
            #         print("saved ",str(i)," with mIoU:",val)
            #         min_val=val
            #         torch.save(model.state_dict(), "alchemy_"+str(iv)+"_"+str(val)[0:6]+"_d12.torch")
        #     #val = evalm(model,fintestloader)
        #     val = evalm(model,fintestloader)
        #     if(val>min_val):
        #         print("saved ",str(i)," with mIoU:",val)
        #         min_val=val
        #         torch.save(model.state_dict(), "alchemy_"+str(iv)+"_"+str(val)[0:6]+".torch")
        #     #torch.save(model.state_dict(), "alchemy_"+str(iv)+".torch")
        # #scheduler.step()  


0 loss: 8596.900390625
1 loss: 8726.4248046875
2 loss: 8644.9306640625
3 loss: 9171.0283203125


In [11]:
ccl = model.state_dict()

for i in dicl.keys():
    if(i[0:4]=="body"):
        dicl[i] = ccl["backbone."+i[5:]]
        print(i) 
    else:
        pass

print()
    
torch.save(dicl,"new_resnet2.0.torch")

body.conv1.weight
body.bn1.weight
body.bn1.bias
body.bn1.running_mean
body.bn1.running_var
body.bn1.num_batches_tracked
body.layer1.0.conv1.weight
body.layer1.0.bn1.weight
body.layer1.0.bn1.bias
body.layer1.0.bn1.running_mean
body.layer1.0.bn1.running_var
body.layer1.0.bn1.num_batches_tracked
body.layer1.0.conv2.weight
body.layer1.0.bn2.weight
body.layer1.0.bn2.bias
body.layer1.0.bn2.running_mean
body.layer1.0.bn2.running_var
body.layer1.0.bn2.num_batches_tracked
body.layer1.0.conv3.weight
body.layer1.0.bn3.weight
body.layer1.0.bn3.bias
body.layer1.0.bn3.running_mean
body.layer1.0.bn3.running_var
body.layer1.0.bn3.num_batches_tracked
body.layer1.0.downsample.0.weight
body.layer1.0.downsample.1.weight
body.layer1.0.downsample.1.bias
body.layer1.0.downsample.1.running_mean
body.layer1.0.downsample.1.running_var
body.layer1.0.downsample.1.num_batches_tracked
body.layer1.1.conv1.weight
body.layer1.1.bn1.weight
body.layer1.1.bn1.bias
body.layer1.1.bn1.running_mean
body.layer1.1.bn1.running_